In [7]:
!pip install tensorflow


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install pandas scikit-learn numpy joblib


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

df = pd.read_csv("verixsoft_chat_dataset.csv")
df.head()


,ID,Intent,User_Query,System_Instruction,Ideal_Response,Source
0,GEN_001,Identity,Who are you?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
1,GEN_002,Identity,What is Verixsoft?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
2,GEN_003,Identity,Introduce yourself.,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
3,GEN_004,Identity,What company do you represent?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
4,GEN_005,Identity,Are you a real person or a bot?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com


In [2]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  175 non-null    object
 1   Intent              175 non-null    object
 2   User_Query          175 non-null    object
 3   System_Instruction  175 non-null    object
 4   Ideal_Response      175 non-null    object
 5   Source              175 non-null    object
dtypes: object(6)
memory usage: 8.3+ KB


In [3]:
df = df.dropna(subset=["User_Query", "Intent", "Ideal_Response"])

df["User_Query"] = df["User_Query"].str.lower().str.strip()
df["Intent"] = df["Intent"].str.strip()

df.head()


,ID,Intent,User_Query,System_Instruction,Ideal_Response,Source
0,GEN_001,Identity,who are you?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
1,GEN_002,Identity,what is verixsoft?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
2,GEN_003,Identity,introduce yourself.,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
3,GEN_004,Identity,what company do you represent?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com
4,GEN_005,Identity,are you a real person or a bot?,State Verixsoft identity.,"I am the intelligent assistant for Verixsoft, ...",verixsoft.com


In [4]:
from sklearn.preprocessing import LabelEncoder
import joblib

intent_encoder = LabelEncoder()
df["intent_label"] = intent_encoder.fit_transform(df["Intent"])

df[["Intent", "intent_label"]].drop_duplicates()


,Intent,intent_label
0,Identity,28
5,Company,13
9,Location,32
13,Phone,41
16,Email,23
...,...,...
169,Source_Control,54
170,Code_Transfer,11
171,Schedule,52
172,Content,15


In [5]:
joblib.dump(intent_encoder, "intent_encoder.pkl")


['intent_encoder.pkl']

In [7]:
counts = y.value_counts()
valid_classes = counts[counts >= 2].index

df = df[df["intent_label"].isin(valid_classes)]


In [9]:
X = df["User_Query"]
y = df["intent_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.4,
    random_state=42,
    stratify=y
)


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

VOCAB_SIZE = 5000
MAX_LEN = 20

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")

joblib.dump(tokenizer, "tokenizer.pkl")


['tokenizer.pkl']

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(VOCAB_SIZE, 64, input_length=MAX_LEN),
    LSTM(64),
    Dropout(0.3),
    Dense(len(intent_encoder.classes_), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


C:\Users\Adeel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(
    X_train_pad,
    y_train,
    epochs=25,
    batch_size=8,
    validation_split=0.1
)


Epoch 1/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.0000e+00 - loss: 4.2056 - val_accuracy: 0.0000e+00 - val_loss: 4.1989
Epoch 2/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0465 - loss: 4.1842 - val_accuracy: 0.0000e+00 - val_loss: 4.1904
Epoch 3/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0233 - loss: 4.1413 - val_accuracy: 0.0000e+00 - val_loss: 4.1877
Epoch 4/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0233 - loss: 4.0842 - val_accuracy: 0.0000e+00 - val_loss: 4.2803
Epoch 5/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0465 - loss: 4.0409 - val_accuracy: 0.0000e+00 - val_loss: 4.3396
Epoch 6/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0233 - loss: 3.9833 - val_accuracy: 0.0000e+00 - val_loss: 4.3417
Epoch 7/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0233 - loss: 3.9470 - val_accuracy: 0.0000e+00 - val_loss: 4.3754
Epoch 8/25
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0349 - loss: 3

In [14]:
model.save("intent_model.keras")


In [15]:
import numpy as np

def chatbot_response(text):
    text = text.lower().strip()
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding="post")
    pred = model.predict(pad)
    intent_id = np.argmax(pred)
    intent = intent_encoder.inverse_transform([intent_id])[0]
    return intent


In [16]:
chatbot_response("who are you")
chatbot_response("where is your office")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


'Observability'

In [17]:

chatbot_response("who are you")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


'Stats'

In [19]:
chatbot_response("who are you")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'Stats'

In [27]:
intent_response_map = {
    "greeting": [
        "Hello! How can I help you?"
    ],
    "goodbye": [
        "Goodbye!"
    ],
    "who_you_are": [
        "I am an AI chatbot trained to answer your questions."
    ],
    "stats": [
        "Here are the stats you asked for."
    ]
}


In [28]:
def chatbot_response(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN)

    pred = model.predict(padded)
    intent_id = pred.argmax()

    intent = intent_encoder.inverse_transform([intent_id])[0]
    response = intent_response_map[intent][0]

    return intent, response


In [29]:
chatbot_response("who you are")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


KeyError: 'Backend'

In [30]:
print(intent_encoder.classes_)



['AI_ML' 'Accessibility' 'Analytics' 'Backend' 'Budget' 'CI_CD' 'CMS'
 'Caching' 'Case_Study' 'Change_Control' 'Cloud' 'Code_Transfer'
 'Communication' 'Company' 'Compliance' 'Content' 'Contracts' 'Database'
 'Deliverables' 'Design_System' 'Disambiguation' 'Discounts' 'Ecommerce'
 'Email' 'Environments' 'Experience' 'Governance' 'IP_Ownership'
 'Identity' 'Industry' 'Integrations' 'Localization' 'Location'
 'Migration' 'Mobile_Stack' 'NDA' 'Observability' 'Onboarding' 'PWA'
 'Payments' 'Performance' 'Phone' 'Pipelines' 'Pricing' 'Process'
 'Publishing' 'Reliability' 'Reporting' 'Rescue' 'Risk' 'SEO_Performance'
 'SLA' 'Schedule' 'Security' 'Source_Control' 'Staffing' 'Stats' 'Storage'
 'Support' 'Support_Channel' 'Team' 'Testing' 'Timeline' 'Timezone'
 'Training' 'Warranty' 'Web_Stack']


In [31]:
print(intent_response_map.keys())


dict_keys(['greeting', 'goodbye', 'who_you_are', 'stats'])
